Ok so I am importing all csv files (tables) at the formulas dir: 

In [1]:
import glob
import pandas as pd
import numpy as np
from functools import reduce
import copy

input_formulas = glob.glob('results/formulas/*.csv')

I preprocess them a little and concat them. However, there are duplicate formulas assigned for a bit different "mz" and "RT" values:

In [2]:
DF_SIRIUS= []
for i, formulas in enumerate(input_formulas):
    df= pd.read_csv(formulas, index_col="Unnamed: 0")
    df_rank= df.filter(regex=fr"opt_global_rank")
    df_score=df.filter(regex=fr"Score")
    df_opt=df.filter(regex=fr"opt")
    cols_score= df_score.columns
    cols_opt= df_opt.columns
    for i, row in df_rank.iterrows():
        if row.sum()>=2:
            df.loc[[i],row.index] = np.nan
    df= df.dropna()
    df= df.drop(columns=cols_score)
    df= df.drop(columns= cols_opt)
    df=df.reset_index()
    df= df.drop(columns="index")
    DF_SIRIUS.append(df)
    
df_formulas= pd.concat(DF_SIRIUS, axis=0).sort_values("chemical_formula")
df_formulas_helper= df_formulas.copy(deep=True)
df_formulas= df_formulas.set_index("chemical_formula")
df_formulas

,exp_mass_to_charge,retention_time
chemical_formula,,
C10H10N4O4,251.077489,176.203788
C10H10O2,180.101879,104.087659
C10H10O2,180.101881,101.885481
C10H11N4O2,237.123300,263.080686
C10H11NO2,195.112813,102.741705
...,...,...
C9H9N6O,235.118960,64.496458
C9H9NO2,181.097131,157.762375
C9H9NO2,181.097127,150.060453


I keep the singletons in a separate dataframe for later (to concat them in the end)

In [3]:
df_singletons=df_formulas.reset_index().drop_duplicates(subset="chemical_formula", keep=False)
df_singletons= df_singletons.rename(columns={"chemical_formula": "formulas", "exp_mass_to_charge": "mz", "retention_time": "RT"})
df_singletons= df_singletons.set_index("formulas")
df_singletons

,mz,RT
formulas,,
C10H10N4O4,251.077489,176.203788
C10H11N4O2,237.123300,263.080686
C10H11NO6,259.092443,65.675430
C10H12N,169.085889,516.704356
C10H12N3O2,224.128145,59.713038
...,...,...
C9H7N4O,205.097117,279.260926
C9H7NO2,162.054953,250.272653
C9H8NO,164.093070,69.147407


Here, I create a dictionary with keys = formulas and values = lists of "mz" and "RT" for the repeating formulas only

In [31]:
from itertools import chain
from collections import defaultdict

dict={}

for formula in np.unique(df_formulas.index):
    if len(df_formulas_helper[df_formulas_helper["chemical_formula"]==formula])>=2:
        for i, lists in enumerate((df_formulas_helper[df_formulas_helper["chemical_formula"]==formula]).values.tolist()):
            key, value= lists[0], lists[1:]
            dict.setdefault(key, []).append(value)
print(dict)

{'C10H10O2': [[180.101879031796, 104.08765896], [180.101881373319, 101.885480576]], 'C10H11NO2': [[195.112813305145, 102.741704784], [195.112789627543, 77.195527216], [195.11287685624, 262.142896304], [195.112808611606, 264.19454888], [195.112626918703, 81.59006312], [195.112804222396, 263.687326032], [195.112804336248, 263.534131856], [195.112802163128, 101.73933048], [195.112874852042, 258.386494784], [195.112592576541, 82.8865436], [195.112805933419, 100.557059392], [195.112740786961, 81.713394736], [195.112808685894, 267.321549872], [195.112817762835, 95.580448352], [195.112813140399, 93.949922528], [195.112812525827, 259.308351232], [195.112827788729, 262.664505136], [195.112808186249, 262.762166208], [195.112805556722, 79.403318512], [195.112809511815, 264.268602144], [195.112804984038, 105.82856264]], 'C10H11NO3': [[211.107710317522, 109.954329328], [211.107743304493, 77.195527216], [211.107730601307, 109.225284448], [211.107757130016, 80.60145248], [211.107721820462, 112.870576

I want to merge those "mz" and "RT" values IF the "mz" values have 20 ppm difference (see mass_delta formula) and also the RT values are different by -20s and +20s max. However, this is straightforward to me when the length of the lists is 2 (duplicates). But, when it becomes higher than 2, I m not sure how to handle this comparisons and so I made a different rule. But it works poorly. Can you help me do the same as when the len(values)==2?

In [38]:
from math import isclose

for key, values in enumerate(dict.values()):
    if len(values) == 2:
        mass_delta= (abs(values[0][0]-values[1][0])/values[0][0])*1000000.0
        maxdeltaRT= values[1][1]+20.0
        mindeltaRT= values[1][1]-20.0
        if (mindeltaRT <= values[0][1] <=  maxdeltaRT) &(mass_delta<= 20.0):
            values.pop()
    if len(values) > 2:
        x= len(values)-1
        a= values[0][0]
        A= values[0][1]
        for i, (b, B) in enumerate(zip([v[0] for v in values[1:]], [v[1] for v in values[1:]])):
            #print(i, b, B)
            mass_delta= (abs(a-b)/a)*1000000.0
            maxdeltaRT= A+20.0
            mindeltaRT= A-20.0
            if (mindeltaRT <= B <=  maxdeltaRT) &(mass_delta<= 20.0):
                del values[i] 
print(dict)

{'C10H10O2': [[180.101879031796, 104.08765896]], 'C10H11NO2': [[195.112813305145, 102.741704784], [195.112789627543, 77.195527216], [195.11287685624, 262.142896304], [195.112808611606, 264.19454888], [195.112626918703, 81.59006312], [195.112804222396, 263.687326032], [195.112804336248, 263.534131856]], 'C10H11NO3': [[211.107710317522, 109.954329328], [211.107743304493, 77.195527216]], 'C10H12N2O': [[194.128852118112, 101.692306192]], 'C10H12N2O4': [[242.113555327239, 139.655391264]], 'C10H12O5': [[230.102162188825, 72.408255392]], 'C10H13N3O3': [[224.102977364025, 227.101249968]], 'C10H13NO2': [[180.101878000293, 318.33574976], [180.101890160125, 318.665578512], [180.101889936565, 362.641795728]], 'C10H13NO3': [[196.096805094733, 71.089349392], [213.123346229221, 242.985808208], [213.123327478919, 243.027674768], [213.123350813824, 51.65798568]], 'C10H13NO3S': [[245.095466715257, 114.5727328], [245.095366046942, 157.762375376]], 'C10H13NO4': [[229.118300709627, 49.174366432]], 'C10H14N